In [21]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.metrics import mean_squared_error

import pickle

In [2]:
import mlflow

mlflow.set_tracking_uri('sqlite:///mlflow.db')
mlflow.set_experiment('nyc-taxi-experiment')

2024/05/15 16:22:23 INFO mlflow.tracking.fluent: Experiment with name 'nyc-taxi-experiment' does not exist. Creating a new experiment.


<Experiment: artifact_location='/Users/ivanivanov/Documents/mlops-zoomcamp/Module 2/mlruns/1', creation_time=1715757743095, experiment_id='1', last_update_time=1715757743095, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

## EDA

In [3]:
df = pd.read_parquet('green_tripdata_2021-01.parquet')
df.head()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
0,2,2021-01-01 00:15:56,2021-01-01 00:19:52,N,1.0,43,151,1.0,1.01,5.5,0.5,0.5,0.00,0.0,None,0.3,6.80,2.0,1.0,0.00
1,2,2021-01-01 00:25:59,2021-01-01 00:34:44,N,1.0,166,239,1.0,2.53,10.0,0.5,0.5,2.81,0.0,None,0.3,16.86,1.0,1.0,2.75
2,2,2021-01-01 00:45:57,2021-01-01 00:51:55,N,1.0,41,42,1.0,1.12,6.0,0.5,0.5,1.00,0.0,None,0.3,8.30,1.0,1.0,0.00
3,2,2020-12-31 23:57:51,2021-01-01 00:04:56,N,1.0,168,75,1.0,1.99,8.0,0.5,0.5,0.00,0.0,None,0.3,9.30,2.0,1.0,0.00
4,2,2021-01-01 00:16:36,2021-01-01 00:16:40,N,2.0,265,265,3.0,0.00,-52.0,0.0,-0.5,0.00,0.0,None,-0.3,-52.80,3.0,1.0,0.00


In [4]:
df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

df = df[(df.duration >= 1) & (df.duration <= 60)]

In [5]:
df.head()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,...,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,duration
0,2,2021-01-01 00:15:56,2021-01-01 00:19:52,N,1.0,43,151,1.0,1.01,5.5,...,0.5,0.00,0.0,None,0.3,6.80,2.0,1.0,0.00,3.933333
1,2,2021-01-01 00:25:59,2021-01-01 00:34:44,N,1.0,166,239,1.0,2.53,10.0,...,0.5,2.81,0.0,None,0.3,16.86,1.0,1.0,2.75,8.750000
2,2,2021-01-01 00:45:57,2021-01-01 00:51:55,N,1.0,41,42,1.0,1.12,6.0,...,0.5,1.00,0.0,None,0.3,8.30,1.0,1.0,0.00,5.966667
3,2,2020-12-31 23:57:51,2021-01-01 00:04:56,N,1.0,168,75,1.0,1.99,8.0,...,0.5,0.00,0.0,None,0.3,9.30,2.0,1.0,0.00,7.083333
7,2,2021-01-01 00:26:31,2021-01-01 00:28:50,N,1.0,75,75,6.0,0.45,3.5,...,0.5,0.96,0.0,None,0.3,5.76,1.0,1.0,0.00,2.316667


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 73908 entries, 0 to 76517
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   VendorID               73908 non-null  int64         
 1   lpep_pickup_datetime   73908 non-null  datetime64[us]
 2   lpep_dropoff_datetime  73908 non-null  datetime64[us]
 3   store_and_fwd_flag     38175 non-null  object        
 4   RatecodeID             38175 non-null  float64       
 5   PULocationID           73908 non-null  int64         
 6   DOLocationID           73908 non-null  int64         
 7   passenger_count        38175 non-null  float64       
 8   trip_distance          73908 non-null  float64       
 9   fare_amount            73908 non-null  float64       
 10  extra                  73908 non-null  float64       
 11  mta_tax                73908 non-null  float64       
 12  tip_amount             73908 non-null  float64       
 13  tolls_

In [7]:
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

df.loc[:, categorical] = df.loc[:,categorical].astype(str)

train_dicts = df[categorical + numerical].to_dict(orient='records')

/var/folders/dz/nf0874bd6dx7_2jbvw0z7lcw0000gn/T/ipykernel_91458/417869314.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['43' '166' '41' ... '74' '168' '119']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[:, categorical] = df.loc[:,categorical].astype(str)
/var/folders/dz/nf0874bd6dx7_2jbvw0z7lcw0000gn/T/ipykernel_91458/417869314.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['151' '239' '42' ... '69' '215' '244']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[:, categorical] = df.loc[:,categorical].astype(str)


## Base model

In [8]:
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

In [9]:
target = 'duration'
y_train = df[target].values
y_train[:5]

array([3.93333333, 8.75      , 5.96666667, 7.08333333, 2.31666667])

In [10]:
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [11]:
y_pred = lr.predict(X_train)

In [12]:
mean_squared_error(y_train, y_pred, squared=False) #RMSE
# 9 -> model is wrong on avg 9 mins
# if a trip is actually 30, we say its 21/39

9.838799799829625

## Load validation data

In [13]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [14]:
df_train = read_dataframe('green_tripdata_2021-01.parquet')
df_val = read_dataframe('green_tripdata_2021-02.parquet')

In [15]:
len(df_train), len(df_val)

(73908, 61921)

In [16]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [17]:
categorical = ['PU_DO']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [18]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

## Model with validation data

In [19]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

7.758715204520257

## Export model

In [20]:
with open('models/lin_reg.bin', 'wb') as f:
    pickle.dump((dv, lr), f)

## Use mlflow

In [23]:
with mlflow.start_run():

    mlflow.set_tag('developer', 'ivan')
    mlflow.log_param('train-data-path', 'green_tripdata_2021-01.csv')
    mlflow.log_param('valid-data-path', 'green_tripdata_2021-02.csv')

    alpha = 0.1
    mlflow.log_param('alpha', alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)
    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric('rmse', rmse)

In [24]:
import xgboost as xgb

#hyperopt uses bayesian methods to find hyperparams
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [27]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [28]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag('model', 'xgboost')
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric('rmse', rmse)
    return {'loss': rmse, 'status': STATUS_OK}

In [29]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0), # lr ~ {exp(-3), exp(0)}
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

/Users/ivanivanov/miniconda3/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [16:47:42] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:9.88996                           
[1]	validation-rmse:8.48013                           
[2]	validation-rmse:7.65491                           
[3]	validation-rmse:7.18728                           
[4]	validation-rmse:6.92423                           
[5]	validation-rmse:6.77299                           
[6]	validation-rmse:6.68427                           
[7]	validation-rmse:6.62833                           
[8]	validation-rmse:6.59174                           
[9]	validation-rmse:6.56845                           
[10]	validation-rmse:6.55191                          
[11]	validation-rmse:6.54004                          
[12]	validation-rmse:6.53030                          
[13]	validation-rmse:6.51799                          
[14]	validation-rmse:6.51255                          
[15]	validation-rmse:6.50770                          
[16]	validation-rmse:6.50434                          
[17]	validation-rmse:6.50199                          
[18]	valid

/Users/ivanivanov/miniconda3/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [16:48:11] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[2]	validation-rmse:10.30017                                                   
[3]	validation-rmse:9.81123                                                    
[4]	validation-rmse:9.38407                                                    
[5]	validation-rmse:9.01243                                                    
[6]	validation-rmse:8.68986                                                    
[7]	validation-rmse:8.41117                                                    
[8]	validation-rmse:8.17076                                                    
[9]	validation-rmse:7.96370                                                    
[10]	validation-rmse:7.78552                                                   
[11]	validation-rmse:7.63238                                                   
[12]	validation-rmse:7.50115                                                   
[13]	validation-rmse:7.38826                                                   
[14]	validation-rmse:7.29186            

...

[0]	validation-rmse:10.56457                                                    
[1]	validation-rmse:9.36668                                                     
[2]	validation-rmse:8.50986                                                     
[3]	validation-rmse:7.90987                                                     
[4]	validation-rmse:7.49307                                                     
[5]	validation-rmse:7.20810                                                     
[6]	validation-rmse:7.01213                                                     
[7]	validation-rmse:6.87732                                                     
[8]	validation-rmse:6.78345                                                     
[9]	validation-rmse:6.71864                                                     
[10]	validation-rmse:6.66915                                                    
[11]	validation-rmse:6.63390                                                    
[12]	validation-rmse:6.60283

/Users/ivanivanov/miniconda3/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [17:18:29] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:9.18000                                                     
[1]	validation-rmse:7.69577                                                     
[2]	validation-rmse:7.02681                                                     
[3]	validation-rmse:6.72280                                                     
[4]	validation-rmse:6.58205                                                     
[5]	validation-rmse:6.50920                                                     
[6]	validation-rmse:6.46993                                                     
[7]	validation-rmse:6.44757                                                     
[8]	validation-rmse:6.43053                                                     
[9]	validation-rmse:6.41863                                                     
[10]	validation-rmse:6.41251                                                    
[11]	validation-rmse:6.40829                                                    
[12]	validation-rmse:6.40445

/Users/ivanivanov/miniconda3/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [17:18:44] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:7.89730                                                     
[1]	validation-rmse:6.92017                                                     
[2]	validation-rmse:6.70282                                                     
[3]	validation-rmse:6.63428                                                     
[4]	validation-rmse:6.60620                                                     
[5]	validation-rmse:6.59888                                                     
[6]	validation-rmse:6.59089                                                     
[7]	validation-rmse:6.58350                                                     
[8]	validation-rmse:6.57598                                                     
[9]	validation-rmse:6.57063                                                     
[10]	validation-rmse:6.56550                                                    
[11]	validation-rmse:6.55531                                                    
[12]	validation-rmse:6.55316

## Train a model with the best params (based on rmse)

In [35]:
mlflow.xgboost.autolog(disable=True)

In [30]:
params = {
    'learning_rate': 0.21602898401340823,
    'max_depth': 70,
    'min_child_weight': 1.8408245354629367,
    'objective': 'reg:linear',
    'reg_alpha': 0.0742372788501083,
    'reg_lambda': 0.32828504532779496,
    'seed': 42,
}

# we can log the train info like before, but we can also use autolog
# mlflow.xgboost.autolog(disable=True)

booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )

2024/05/15 17:45:38 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'a8bd736545304ebdbd20500d75bbe8a5', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current xgboost workflow
/Users/ivanivanov/miniconda3/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [17:45:42] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[0]	validation-rmse:10.48712
[1]	validation-rmse:9.24734
[2]	validation-rmse:8.37411
[3]	validation-rmse:7.77345
[4]	validation-rmse:7.36221
[5]	validation-rmse:7.08295
[6]	validation-rmse:6.89389
[7]	validation-rmse:6.76639
[8]	validation-rmse:6.67815
[9]	validation-rmse:6.61452
[10]	validation-rmse:6.56861
[11]	validation-rmse:6.53428
[12]	validation-rmse:6.50975
[13]	validation-rmse:6.49200
[14]	validation-rmse:6.47654
[15]	validation-rmse:6.46379
[16]	validation-rmse:6.45419
[17]	validation-rmse:6.44556
[18]	validation-rmse:6.43769
[19]	validation-rmse:6.43221
[20]	validation-rmse:6.42798
[21]	validation-rmse:6.42532
[22]	validation-rmse:6.42277
[23]	validation-rmse:6.42016
[24]	validation-rmse:6.41921
[25]	validation-rmse:6.41731
[26]	validation-rmse:6.41504
[27]	validation-rmse:6.41298
[28]	validation-rmse:6.41007
[29]	validation-rmse:6.40791
[30]	validation-rmse:6.40612
[31]	validation-rmse:6.40494
[32]	validation-rmse:6.40263
[33]	validation-rmse:6.40086
[34]	validation-rmse:6.

2024/05/15 17:46:24 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.
2024/05/15 17:46:24 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/ivanivanov/miniconda3/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [17:46:24] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified."


## Using mlflow for model management

In [33]:
with mlflow.start_run():

    mlflow.set_tag('developer', 'ivan')
    mlflow.log_param('train-data-path', 'green_tripdata_2021-01.csv')
    mlflow.log_param('valid-data-path', 'green_tripdata_2021-02.csv')

    alpha = 0.1
    mlflow.log_param('alpha', alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)
    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric('rmse', rmse)

    mlflow.log_artifact(local_path='models/lin_reg.bin', 
                        artifact_path='models_pickle')

## Better way of logging models to mlflow

In [44]:
with mlflow.start_run():
    params = {
    'learning_rate': 0.21602898401340823,
    'max_depth': 70,
    'min_child_weight': 1.8408245354629367,
    'objective': 'reg:linear',
    'reg_alpha': 0.0742372788501083,
    'reg_lambda': 0.32828504532779496,
    'seed': 42,
    }

    mlflow.log_params(params)

    booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
    
    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric('rmse', rmse)
    with open('models/preprocess.b', 'wb') as f_out:
        pickle.dump(dv, f_out)

    mlflow.log_artifact('models/preprocess.b', artifact_path='preprocessor')

    mlflow.xgboost.log_model(booster, artifact_path='models_mlflow')


/Users/ivanivanov/miniconda3/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [19:03:31] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[0]	validation-rmse:9.85826
[1]	validation-rmse:8.42384
[2]	validation-rmse:7.59399
[3]	validation-rmse:7.11874
[4]	validation-rmse:6.85000
[5]	validation-rmse:6.69435
[6]	validation-rmse:6.60302
[7]	validation-rmse:6.54386
[8]	validation-rmse:6.50581
[9]	validation-rmse:6.48002
[10]	validation-rmse:6.46347
[11]	validation-rmse:6.44842
[12]	validation-rmse:6.43989
[13]	validation-rmse:6.43252
[14]	validation-rmse:6.42641
[15]	validation-rmse:6.42306
[16]	validation-rmse:6.42011
[17]	validation-rmse:6.41831
[18]	validation-rmse:6.41637
[19]	validation-rmse:6.41387
[20]	validation-rmse:6.41089
[21]	validation-rmse:6.40702
[22]	validation-rmse:6.40499
[23]	validation-rmse:6.40244
[24]	validation-rmse:6.40062
[25]	validation-rmse:6.39887
[26]	validation-rmse:6.39702
[27]	validation-rmse:6.39523
[28]	validation-rmse:6.39206
[29]	validation-rmse:6.39035
[30]	validation-rmse:6.38890
[31]	validation-rmse:6.38765
[32]	validation-rmse:6.38522
[33]	validation-rmse:6.38353
[34]	validation-rmse:6.3

/Users/ivanivanov/miniconda3/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [19:03:47] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
  warnings.warn(smsg, UserWarning)
/Users/ivanivanov/miniconda3/lib/python3.11/site-packages/_distutils_hack/__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
/Users/ivanivanov/miniconda3/lib/python3.11/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutil

## Using a saved model for predictions

In [38]:
logged_model = 'runs:/6c83ec7fe4ec4f5a9c4b0e1ee2fe01f9/models_mlflow'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

/Users/ivanivanov/miniconda3/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [18:32:06] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


In [39]:
loaded_model

mlflow.pyfunc.loaded_model:
  artifact_path: models_mlflow
  flavor: mlflow.xgboost
  run_id: 6c83ec7fe4ec4f5a9c4b0e1ee2fe01f9

In [ ]:
xgboost_model = mlflow.xgboost.load_model(logged_model)

In [42]:
# same as before
y_pred = xgboost_model.predict(valid)

In [43]:
y_pred[:10]

array([14.335526 ,  6.7993455, 13.805621 , 24.455435 ,  9.881113 ,
       17.21585  , 10.113814 ,  7.797032 ,  8.875645 , 19.383533 ],
      dtype=float32)

## Use mlflow client

In [45]:
from mlflow.tracking import MlflowClient

MLFLOW_TRACKING_URI = 'sqlite:///mlflow.db'

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [47]:
client.search_experiments()

[<Experiment: artifact_location='/Users/ivanivanov/Documents/mlops-zoomcamp/Module 2/mlruns/1', creation_time=1715757743095, experiment_id='1', last_update_time=1715757743095, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1715757464080, experiment_id='0', last_update_time=1715757464080, lifecycle_stage='active', name='Default', tags={}>]

In [48]:
client.create_experiment(name='my-cool-experiment')

'2'

In [52]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids=['1'], # nyc-taxi-experiment
    filter_string='',
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=['metrics.rmse ASC']
)

In [58]:
for run in runs:
    print(f"{run.info.run_id=}, {run.data.metrics['rmse']=}")

run.info.run_id='6c83ec7fe4ec4f5a9c4b0e1ee2fe01f9', run.data.metrics['rmse']=6.311489169647269
run.info.run_id='2a0eab3845a64da1a486778c0904c02c', run.data.metrics['rmse']=6.311489169647269
run.info.run_id='e1ea474e5d6b46e4a68e456cec2ec74a', run.data.metrics['rmse']=6.311489169647269
run.info.run_id='2f9de23de5a34138aa3f8c107d446b26', run.data.metrics['rmse']=6.316696115916369
run.info.run_id='4e6e9b19d5ec4910a563757ea418e866', run.data.metrics['rmse']=6.317329937700502


In [59]:
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [60]:
model_uri = '4e6e9b19d5ec4910a563757ea418e866'
mlflow.register_model(model_uri=model_uri, name='nyc-taxi-regressor')

Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
Created version '3' of model 'nyc-taxi-regressor'.


<ModelVersion: aliases=[], creation_timestamp=1715768842192, current_stage='None', description=None, last_updated_timestamp=1715768842192, name='nyc-taxi-regressor', run_id=None, run_link=None, source='4e6e9b19d5ec4910a563757ea418e866', status='READY', status_message=None, tags={}, user_id=None, version=3>

In [62]:
client.search_registered_models()

[<RegisteredModel: aliases={'challenger': 2, 'champion': 1}, creation_timestamp=1715767290697, description='NYC taxi trip prediction', last_updated_timestamp=1715768842192, latest_versions=[<ModelVersion: aliases=[], creation_timestamp=1715768842192, current_stage='None', description=None, last_updated_timestamp=1715768842192, name='nyc-taxi-regressor', run_id=None, run_link=None, source='4e6e9b19d5ec4910a563757ea418e866', status='READY', status_message=None, tags={}, user_id=None, version=3>], name='nyc-taxi-regressor', tags={}>]

In [63]:
client.get_latest_versions(name='nyc-taxi-regressor')

/var/folders/dz/nf0874bd6dx7_2jbvw0z7lcw0000gn/T/ipykernel_91458/613867140.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.12.2/model-registry.html#migrating-from-stages
  client.get_latest_versions(name='nyc-taxi-regressor')


[<ModelVersion: aliases=[], creation_timestamp=1715768842192, current_stage='None', description=None, last_updated_timestamp=1715768842192, name='nyc-taxi-regressor', run_id=None, run_link=None, source='4e6e9b19d5ec4910a563757ea418e866', status='READY', status_message=None, tags={}, user_id=None, version=3>]

In [74]:
client.transition_model_version_stage(name='nyc-taxi-regressor', version='1', stage='Production')

/var/folders/dz/nf0874bd6dx7_2jbvw0z7lcw0000gn/T/ipykernel_91458/2380073287.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.12.2/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(name='nyc-taxi-regressor', version='1', stage='Production')


<ModelVersion: aliases=['production'], creation_timestamp=1715767290733, current_stage='Production', description='', last_updated_timestamp=1715770034387, name='nyc-taxi-regressor', run_id='6c83ec7fe4ec4f5a9c4b0e1ee2fe01f9', run_link='', source=('/Users/ivanivanov/Documents/mlops-zoomcamp/Module '
 '2/mlruns/1/6c83ec7fe4ec4f5a9c4b0e1ee2fe01f9/artifacts/models_mlflow'), status='READY', status_message=None, tags={'lr': '0.3'}, user_id=None, version=1>

## Promote a model to production

In [64]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical+numerical].to_dict(orient='records')
    return dv.transform(train_dicts)

def test_model(stage, X_test, y_test, name='nyc-taxi-regressor'):
    model = mlflow.pyfunc.load_model(f'models:/{name}/{stage}')
    y_pred = model.predict(X_test)
    return {'rmse': mean_squared_error(y_test, y_pred, squared=False)}

In [65]:
df = read_dataframe('green_tripdata_2021-03.parquet')

In [66]:
client.download_artifacts(run_id=model_uri, path='preprocessor', dst_path='.')

'/Users/ivanivanov/Documents/mlops-zoomcamp/Module 2/preprocessor'

In [67]:
with open('preprocessor/preprocess.b', 'rb') as f_in:
    dv = pickle.load(f_in)

In [68]:
X_test = preprocess(df, dv)

In [69]:
target = 'duration'
y_test = df[target].values

In [75]:
%time test_model(stage='Production', X_test=X_test, y_test=y_test)

/Users/ivanivanov/miniconda3/lib/python3.11/site-packages/mlflow/store/artifact/utils/models.py:32: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.12.2/model-registry.html#migrating-from-stages
  latest = client.get_latest_versions(name, None if stage is None else [stage])
/Users/ivanivanov/miniconda3/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [19:48:33] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


CPU times: user 13.8 s, sys: 132 ms, total: 14 s
Wall time: 1.93 s


{'rmse': 6.261388258640052}